In [1]:
%load_ext autoreload
%autoreload 2

import torch
from diffusers.models import ControlNetModel
from diffusers.schedulers import DPMSolverMultistepScheduler

from text3d2video.utilities.camera_placement import front_facing_extrinsics
from text3d2video.utilities.mesh_processing import normalize_meshes
from text3d2video.rendering import render_depth_map
from text3d2video.utilities.video_util import pil_frames_to_clip
from pytorch3d.renderer import FoVOrthographicCameras
from pytorch3d.io import load_obj, load_objs_as_meshes

torch.set_grad_enabled(False)

mesh_path = "data/meshes/mixamo-human.obj"
device = "cuda"

# read mesh
verts, faces, aux = load_obj(mesh_path)
verts_uvs = aux.verts_uvs.to(device)
faces_uvs = faces.textures_idx.to(device)

mesh = load_objs_as_meshes([mesh_path], device=device)
mesh = normalize_meshes(mesh)

s = 1.8
dist = 1

In [35]:
from text3d2video.artifacts.anim_artifact import AnimationArtifact
from text3d2video.utilities.ipython_utils import display_vid

n_frames = 5
angles = torch.linspace(0, 360, n_frames)
xs = torch.linspace(-0.5, 0.5, n_frames)
R, T = front_facing_extrinsics(degrees=angles, zs=dist)

frame_cams = FoVOrthographicCameras(R=R, T=T, device="cuda", scale_xyz=[(s, s, s)])
frame_meshes = mesh.extend(len(frame_cams))


animation = AnimationArtifact.from_wandb_artifact_tag('rumba:latest')
frame_indices = animation.frame_indices(n_frames)
frame_cams, frame_meshes = animation.load_frames(frame_indices)

depths = render_depth_map(frame_meshes, frame_cams)

display_vid(pil_frames_to_clip(depths))

In [36]:
from text3d2video.pipelines.generative_rendering_pipeline import GenerativeRenderingPipeline

device = torch.device("cuda")
dtype = torch.float16

sd_repo = "runwayml/stable-diffusion-v1-5"
controlnet_repo = "lllyasviel/control_v11f1p_sd15_depth"

controlnet = ControlNetModel.from_pretrained(controlnet_repo, torch_dtype=dtype).to(
    device
)

pipe: GenerativeRenderingPipeline = GenerativeRenderingPipeline.from_pretrained(
    sd_repo, controlnet=controlnet, torch_dtype=dtype
).to(device)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  warnings.warn(



In [44]:
from text3d2video.artifacts.gr_data import GrSaveConfig
from text3d2video.generative_rendering.configs import GenerativeRenderingConfig
from text3d2video.noise_initialization import UVNoiseInitializer

module_paths = [
    # "down_blocks.0.attentions.0.transformer_blocks.0.attn1",
    # "down_blocks.0.attentions.1.transformer_blocks.0.attn1",
    # "down_blocks.1.attentions.0.transformer_blocks.0.attn1",
    # "down_blocks.1.attentions.1.transformer_blocks.0.attn1",
    # "down_blocks.2.attentions.0.transformer_blocks.0.attn1",
    # "down_blocks.2.attentions.1.transformer_blocks.0.attn1",
    # "mid_block.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.1.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.2.attentions.2.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.0.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.1.transformer_blocks.0.attn1",
    "up_blocks.3.attentions.2.transformer_blocks.0.attn1",
]

gr_config = GenerativeRenderingConfig(
    seed=1,
    resolution=512,
    do_pre_attn_injection=True,
    do_post_attn_injection=True,
    feature_blend_alpha=1.0,
    attend_to_self_kv=False,
    mean_features_weight=1.0,
    chunk_size=5,
    num_keyframes=3,
    num_inference_steps=10,
    guidance_scale=7.5,
    controlnet_conditioning_scale=1.5,
    module_paths=module_paths,
)

gr_save_cfg = GrSaveConfig(
    enabled=True,
    n_frames=5,
    n_timesteps=5,
    save_latents=False,
    save_q=True,
    save_k=True,
    save_v=True,
    out_artifact="rumba",
    module_paths=module_paths,
    save_kf_post_attn=True,
    save_aggregated_features=True,
    save_feature_images=True
)

prompt = "Deadpool, simple blank background"

noise_initializer = UVNoiseInitializer()

generator = torch.Generator(device='cuda')
generator.manual_seed(0)

video_frames = pipe(
    prompt,
    frame_meshes,
    frame_cams,
    verts_uvs,
    faces_uvs,
    generative_rendering_config=gr_config,
    noise_initializer=noise_initializer,
    gr_save_config=gr_save_cfg,
    generator=generator,
)

art = pipe.gr_data_artifact

100%|██████████| 10/10 [00:14<00:00,  1.44s/it]


In [ ]:
vid_frames = video_frames[0:len(frame_cams)]
aggr_frames = video_frames[len(frame_cams):]

display_vid(pil_frames_to_clip(vid_frames), resolution=512)

: 

In [20]:

from text3d2video.utilities.h5_util import print_datasets

print_datasets(art.h5_file_path())